# Things done these two weeks (2018-10-18)

## Implemented more functions to compute and load the features

In [1]:
from piano_reduction import compute_features as cf
x_train, y_train = cf.load_features(2)
x_train.head()

,active_rhythm,bass_line,entrance_effect,harmony,highest,in_chord,lowest,occurrence,offset_value,onset_after_rest,pitch_distance,rhythm_variety,strong_beats,sustained_rhythm,vertical_doubling
0,1,0,0.0,1,1,0,0,1,0.5,1,7.0,1,0,1,1
1,1,0,0.0,1,0,0,0,1,0.5,1,5.0,1,0,1,1
2,1,1,0.0,1,0,0,1,1,0.5,1,17.0,1,0,1,1
3,1,0,0.5,1,1,0,0,1,1.0,0,7.0,0,1,1,1
4,1,0,0.5,1,0,0,0,1,1.0,0,5.0,0,1,1,1


In [2]:
cf.feature_count()

15

#### Convert data to sequence data for LSTM

In [3]:
x_train, y_train = cf.generate_data([cf.load_features(2)], length = 20)
x_train.shape, y_train.shape

((1515, 20, 15), (1515, 20, 1))

#### Convert data to one-hot vector for LSTM

In [4]:
from piano_reduction import tools as pr
res = pr.load_merged_df(2)
x_train, y_train = cf.generate_data_2([(cf.df_to_onehot(res[0], 'x_train'), cf.df_to_onehot(res[0], 'y_train'))])
x_train.shape, y_train.shape

((359, 20, 128), (359, 20, 128))

## Tried using Keras NN and LSTM

In [5]:
from keras.models import Sequential
from keras.layers.core import Dense, Dropout, Activation
from keras.layers.embeddings import Embedding
from keras.layers.recurrent import LSTM
from keras import backend as K
from keras.callbacks import BaseLogger
from keras.callbacks import EarlyStopping
def f1(y_true, y_pred):
    def recall(y_true, y_pred):
        true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
        possible_positives = K.sum(K.round(K.clip(y_true, 0, 1)))
        recall = true_positives / (possible_positives + K.epsilon())
        return recall

    def precision(y_true, y_pred):
        true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
        predicted_positives = K.sum(K.round(K.clip(y_pred, 0, 1)))
        precision = true_positives / (predicted_positives + K.epsilon())
        return precision
    precision = precision(y_true, y_pred)
    recall = recall(y_true, y_pred)
    return 2*((precision*recall)/(precision+recall+K.epsilon()))

early_stop = EarlyStopping(monitor='val_acc', min_delta=0, patience=5, verbose=0, restore_best_weights=True)

Using TensorFlow backend.


### Dense Layers

In [6]:
nn = Sequential()
nn.add(Dense(50, input_shape=(None, cf.feature_count())))
nn.add(Activation('sigmoid'))
nn.add(Dense(1))
nn.add(Activation('sigmoid'))
nn.compile(optimizer='rmsprop',
              loss='binary_crossentropy',
              metrics=['accuracy', f1])
nn.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_1 (Dense)              (None, None, 50)          800       
_________________________________________________________________
activation_1 (Activation)    (None, None, 50)          0         
_________________________________________________________________
dense_2 (Dense)              (None, None, 1)           51        
_________________________________________________________________
activation_2 (Activation)    (None, None, 1)           0         
Total params: 851
Trainable params: 851
Non-trainable params: 0
_________________________________________________________________


In [52]:
x_train, y_train = cf.generate_data([cf.load_features(7, cosi=True)], length = 1)
x_test, y_test = cf.generate_data([cf.load_features(2)], length = 1)
nn.fit(x_train, y_train, epochs=50, batch_size=32, validation_data=(x_test, y_test), callbacks=[early_stop])

Train on 3843 samples, validate on 1534 samples
Epoch 1/50
3843/3843 [==============================] - 0s 57us/step - loss: 0.5965 - acc: 0.6825 - f1: 0.7522 - val_loss: 0.7019 - val_acc: 0.6102 - val_f1: 0.7064
Epoch 2/50
3843/3843 [==============================] - 0s 53us/step - loss: 0.5923 - acc: 0.6872 - f1: 0.7549 - val_loss: 0.6999 - val_acc: 0.6330 - val_f1: 0.7081
Epoch 3/50
3843/3843 [==============================] - 0s 46us/step - loss: 0.5902 - acc: 0.6875 - f1: 0.7551 - val_loss: 0.6790 - val_acc: 0.6349 - val_f1: 0.6774
Epoch 4/50
3843/3843 [==============================] - 0s 51us/step - loss: 0.5882 - acc: 0.6870 - f1: 0.7510 - val_loss: 0.6903 - val_acc: 0.6415 - val_f1: 0.6894
Epoch 5/50
3843/3843 [==============================] - 0s 57us/step - loss: 0.5865 - acc: 0.6893 - f1: 0.7507 - val_loss: 0.6916 - val_acc: 0.6415 - val_f1: 0.7061
Epoch 6/50
3843/3843 [==============================] - 0s 51us/step - loss: 0.5844 - acc: 0.6885 - f1: 0.7511 - val_loss: 0.69

### LSTM

In [8]:
lstm = Sequential()
lstm.add(LSTM(30, input_shape=(None, cf.feature_count()), return_sequences=True))
lstm.add(Dropout(0.2))
lstm.add(Dense(1))
lstm.add(Activation('sigmoid'))
lstm.compile(optimizer='rmsprop',
              loss='binary_crossentropy',
              metrics=['accuracy', f1])
lstm.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm_1 (LSTM)                (None, None, 30)          5520      
_________________________________________________________________
dropout_1 (Dropout)          (None, None, 30)          0         
_________________________________________________________________
dense_3 (Dense)              (None, None, 1)           31        
_________________________________________________________________
activation_3 (Activation)    (None, None, 1)           0         
Total params: 5,551
Trainable params: 5,551
Non-trainable params: 0
_________________________________________________________________


In [9]:
for length in [50]:
    x_train, y_train = cf.generate_data([cf.load_features(7, cosi=True)], length = length)
    x_test, y_test = cf.generate_data([cf.load_features(2)], length = length)
    lstm.fit(x_train, y_train, epochs=50, batch_size=32, validation_data=(x_test, y_test), callbacks=[early_stop])

Train on 3794 samples, validate on 1485 samples
Epoch 1/50
3794/3794 [==============================] - 5s 1ms/step - loss: 0.6736 - acc: 0.5805 - f1: 0.6296 - val_loss: 0.6820 - val_acc: 0.5683 - val_f1: 0.6957
Epoch 2/50
3794/3794 [==============================] - 5s 1ms/step - loss: 0.6351 - acc: 0.6466 - f1: 0.7414 - val_loss: 0.6961 - val_acc: 0.5887 - val_f1: 0.6907
Epoch 3/50
3794/3794 [==============================] - 5s 1ms/step - loss: 0.6132 - acc: 0.6716 - f1: 0.7539 - val_loss: 0.7201 - val_acc: 0.5674 - val_f1: 0.6762
Epoch 4/50
3794/3794 [==============================] - 5s 1ms/step - loss: 0.5925 - acc: 0.6891 - f1: 0.7628 - val_loss: 0.7536 - val_acc: 0.5633 - val_f1: 0.6835
Epoch 5/50
3794/3794 [==============================] - 4s 1ms/step - loss: 0.5755 - acc: 0.7035 - f1: 0.7696 - val_loss: 0.7805 - val_acc: 0.5630 - val_f1: 0.6804
Epoch 6/50
3794/3794 [==============================] - 4s 1ms/step - loss: 0.5603 - acc: 0.7145 - f1: 0.7773 - val_loss: 0.7898 - v

### Results

In [53]:
from piano_reduction import tools as pr
import numpy as np
def get_y_pred(model, df, x_test):
    tmp = model.predict(x_test)
    y = tmp[0,:,0].tolist()
    for i in range(1, tmp.shape[0]):
        y.append(tmp[i,-1,0])
    pred = (np.array(y) > 0.5) * 1
    tmp = []
    count = 0
    for i in df.index:
        if df.loc[i, 'x_train'] == 0:
            tmp.append(0)
        else:
            tmp.append(pred[count])
            count += 1
    return tmp

res = pr.load_merged_df(2)
from sklearn import metrics
print('%10s %11s %11s' % ('', 'Accuracy', 'F1 Score'))
for name, model in [('NN', nn), ('LSTM', lstm)]:
    print('%10s %.9f %.9f' % (name, 
                              metrics.accuracy_score(res[0]['y_train'], get_y_pred(model, res[0], x_test)), 
                              metrics.f1_score(res[0]['y_train'], get_y_pred(model, res[0], x_test))))

              Accuracy    F1 Score
        NN 0.575449102 0.650910881
      LSTM 0.563473054 0.687258687


In [54]:
res = pr.load_merged_df(2)
res[0]['y_pred'] = get_y_pred(lstm, res[0], x_test)
score = pr.df_to_scores(res[0], res[1], res[2], col=['x_train', 'y_pred', 'y_train'])

In [55]:
score.show('musicxml')

### Problems
Only input the notes sequentially

Cannot handle notes with same offset well

## Onehot encoding

In [18]:
from keras.models import load_model
import imp
imp.reload(cf)
model = load_model('models/LSTM_onehot.h5', custom_objects={'f1': f1})
res = pr.load_merged_df(2)
x_train, y_train = cf.generate_data_2([(cf.df_to_onehot(res[0], col='x_train'), cf.df_to_onehot(res[0], col='y_train'))], length=20)
x_train.shape, y_train.shape

((359, 20, 128), (359, 20, 128))

In [23]:
lstm2 = Sequential()
lstm2.add(LSTM(128, input_shape=(None, 128), return_sequences=True))
lstm2.add(Dropout(0.2))
lstm2.add(Activation('sigmoid'))
lstm2.compile(optimizer='rmsprop',
              loss='binary_crossentropy',
              metrics=['accuracy', f1])
lstm2.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm_4 (LSTM)                (None, None, 128)         131584    
_________________________________________________________________
dropout_4 (Dropout)          (None, None, 128)         0         
_________________________________________________________________
activation_6 (Activation)    (None, None, 128)         0         
Total params: 131,584
Trainable params: 131,584
Non-trainable params: 0
_________________________________________________________________


In [20]:
samples = []
for i in range(-12, 13):
    res = pr.load_cosi(7)
    res[0]['ps'] += i
    samples.append((cf.df_to_onehot(res[0], col='x_train'), cf.df_to_onehot(res[0], col='y_train')))
res = pr.load_merged_df(2)
test_sample = [(cf.df_to_onehot(res[0], col='x_train'), cf.df_to_onehot(res[0], col='y_train'))]

In [25]:
for i in [1, 3]:
    x_train, y_train = cf.generate_data_2(samples, length=i)
    x_test, y_test = cf.generate_data_2(test_sample, length=i)
    print(i, x_train.shape, y_train.shape)
    lstm2.fit(x_train, y_train, epochs=10, batch_size=32, callbacks=[early_stop], validation_data=(x_test, y_test))

1 (33525, 1, 128) (33525, 1, 128)
Train on 33525 samples, validate on 378 samples
Epoch 1/10
33525/33525 [==============================] - 3s 100us/step - loss: 0.4091 - acc: 0.9834 - f1: 0.0817 - val_loss: 0.3951 - val_acc: 0.9796 - val_f1: 0.0872
Epoch 2/10
33525/33525 [==============================] - 3s 100us/step - loss: 0.4091 - acc: 0.9835 - f1: 0.0828 - val_loss: 0.3951 - val_acc: 0.9796 - val_f1: 0.0872
Epoch 3/10
33525/33525 [==============================] - 3s 101us/step - loss: 0.4090 - acc: 0.9835 - f1: 0.0819 - val_loss: 0.3951 - val_acc: 0.9796 - val_f1: 0.0872
Epoch 4/10
33525/33525 [==============================] - 3s 102us/step - loss: 0.4090 - acc: 0.9835 - f1: 0.0838 - val_loss: 0.3951 - val_acc: 0.9796 - val_f1: 0.0872
Epoch 5/10
33525/33525 [==============================] - 3s 104us/step - loss: 0.4089 - acc: 0.9835 - f1: 0.0834 - val_loss: 0.3951 - val_acc: 0.9797 - val_f1: 0.0907
Epoch 6/10
33525/33525 [==============================] - 4s 106us/step - loss

In [59]:
def predict_to_y(model, x_test):
    tmp = model.predict(x_test)
    y = tmp[0,:,:].tolist()
    for i in range(1, tmp.shape[0]):
        y.append(tmp[i,-1,:])
    return np.array(y)

from keras.models import load_model
model = load_model('models/LSTM_onehot_3.h5', custom_objects={'f1': f1})
res = pr.load_merged_df(3)
x_test, y_test = cf.generate_data_2([(cf.df_to_onehot(res[0], col='x_train'), 
                                        cf.df_to_onehot(res[0], col='y_train'))], length=2)
res2 = cf.onehot_to_df(res[0], (predict_to_y(model, x_test) >= 0.5) * 1)
from sklearn import metrics
print('Accuracy', metrics.accuracy_score(res2['y_train'], res2['y_pred']))
print('F1 Score', metrics.f1_score(res2['y_train'], res2['y_pred']))
res2.head()

Accuracy 0.5174009943425338
F1 Score 0.5546590729314982


,measure,offset,part,ps,tie,x_train,color,y_train,duration,pitch,y_pred
0,0.0,0,2,72.0,<music21.tie.Tie start>,1,#605050,1,<music21.duration.Duration 2.0>,C5,1
1,0.0,0,2,76.0,<music21.tie.Tie start>,1,#803030,1,<music21.duration.Duration 2.0>,E5,1
2,0.0,0,4,64.0,<music21.tie.Tie start>,1,#803030,1,<music21.duration.Duration 2.0>,E4,0
3,0.0,0,4,69.0,<music21.tie.Tie start>,1,#005000,1,<music21.duration.Duration 2.0>,A4,0
4,0.0,0,6,57.0,<music21.tie.Tie start>,1,#005000,1,<music21.duration.Duration 2.0>,A3,0


In [57]:
pr.df_to_scores(res2, res[1], res[2], col = ['x_train', 'y_pred', 'y_train']).show('musicxml')

In [ ]:
music21.converter.parse('meetings/20181018/o_0003_with_onehot.musicxml').show('musicxml')

### Problems
Very slow to train

The results do not really match with training output

Lose all features

## These results may be representing the similarity of the styles only

In [35]:
from keras.models import clone_model
from tqdm import tqdm
models = {}
x_test, y_test = cf.generate_data([cf.load_features(2)], length = 50)
for i in tqdm(range(1, 2)):
    x_train, y_train = cf.generate_data([cf.load_features(i, cosi=True)], length = 50)
    mname = 'NN %d' % i
    models[mname] = clone_model(nn)
    models[mname].compile(optimizer='rmsprop', loss='binary_crossentropy', metrics=['accuracy', f1])
    models[mname].fit(x_train, y_train, epochs=50, batch_size=32, validation_data=(x_test, y_test), callbacks=[early_stop], verbose=0)
    mname = 'LSTM %d' % i
    models[mname] = clone_model(lstm)
    models[mname].compile(optimizer='rmsprop', loss='binary_crossentropy', metrics=['accuracy', f1])
    models[mname].fit(x_train, y_train, epochs=50, batch_size=32, validation_data=(x_test, y_test), callbacks=[early_stop], verbose=0)
    
for name, model in models.items():
    model.save('models/%s.h5' % name)

100%|██████████| 1/1 [04:06<00:00, 246.12s/it]


In [40]:
res = pr.load_merged_df(2)
from sklearn import metrics
from keras.models import load_model
print('%10s %11s %11s' % ('', 'Accuracy', 'F1 Score'))
models = [('NN %d' % i) for i in range(1, 10)] + [('LSTM %d' % i) for i in range(1, 10)]
for i, name in enumerate(models):
    model = load_model('models/%s.h5' % name, custom_objects={'f1': f1})
    print('%10s %.9f %.9f' % (name, 
                              metrics.accuracy_score(res[0]['y_train'], get_y_pred(model, res[0], x_test)), 
                              metrics.f1_score(res[0]['y_train'], get_y_pred(model, res[0], x_test))))
    res[0]['y_pred_%s' % name] = get_y_pred(model, res[0], x_test)

              Accuracy    F1 Score
      NN 1 0.542514970 0.622902270
      NN 2 0.575449102 0.681061628
      NN 3 0.557485030 0.654834190
      NN 4 0.580239521 0.682661838
      NN 5 0.580239521 0.687750557
      NN 6 0.564071856 0.684575390
      NN 7 0.587425150 0.690615177
      NN 8 0.521556886 0.553381778
      NN 9 0.573053892 0.677812924
    LSTM 1 0.532934132 0.585547290
    LSTM 2 0.553892216 0.596643205
    LSTM 3 0.580838323 0.656862745
    LSTM 4 0.559281437 0.687074830
    LSTM 5 0.556886228 0.669642857
    LSTM 6 0.552095808 0.652416357
    LSTM 7 0.574850299 0.663507109
    LSTM 8 0.537125749 0.594651285
    LSTM 9 0.552694611 0.675076120


In [47]:
score_2 = pr.df_to_scores(res[0], res[1], res[2], col=['y_train', 'y_pred_LSTM 1', 'y_pred_LSTM 2', 'y_pred_LSTM 3'])

In [58]:
score_2.show('musicxml')

## Studied Conditional Random Field

Will continue to use the package **pystruct**

All packages only support sequential CRF only

May not be able to handle notes at the same time as well